In [ ]:
client_id = 'o09u9jcft4mpca3e0n0hofaq0'
client_secret ='1qficodlgj1g2d7p1ltsn3sbmj39t74n3hokfk98re7oqn1nm1do'

In [ ]:
import requests
import base64
def get_access_token():
    params = {
    'grant_type': 'client_credentials'
    }
    basic = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': f'Basic {basic}'
    }
    response = requests.post('https://auth.zmaps-api.com/oauth2/token',
    data=params, headers=headers)
    return response.json()
token = get_access_token()["access_token"]
token

In [ ]:
def fetch_ac_premium_results(word, token):
    params = {
    'word': word,
    'use_multi_addr': 'true',
    'use_kana': 'true',
    'use_pastmap': 'true',
    'use_bluemap': 'true',
    'datum': 'JGD'
    }
    api_key = 'w9d8hx2Njt3ndWK1B5cXL2JBNfSfpbwL4qjcKY4m'
    headers = {
    'x-api-key': api_key,
    'Authorization': f'Bearer {token}'
    }

    response = requests.get('https://web.zmaps-api.com/data-coding/ac_premium', params=params, headers=headers)

    return response.json()

In [ ]:
def add_column(df,new_column):
    token_result = get_access_token()
    #データを80件ずつ処理する
    batch_size = 80
    total_rows = len(df)
    all_items = []

    # 新しい列を作成（初期値はNaN）
    df[new_column] = pd.NA

    #80個ずつdf["full_adress"]から値を取り出して配列に格納
    for i in range(0, total_rows, batch_size):
        batch = df.iloc[i:i + batch_size]["full_address"].tolist()
        input_address = ", ".join(batch)

        ac_premium_result = fetch_ac_premium_results(input_address,token_result['access_token'])
        items = ac_premium_result['result']['item']
        addresses = [item[new_column] for item in items]
        all_items.extend(addresses)
        print(f"バッチ {i//batch_size}: {len(batch)}件処理")

    # 取得したデータを順番に格納
        for idx, code in enumerate(all_items):
            if idx < len(df):
                df.iloc[idx, df.columns.get_loc(new_column)] = code
    return df

#使用例
#引数にデータフレームと新しく追加したいカラムを指定
#https://developers.zmaps-api.com/v20/reference/webAPI/ac_premium.html を参照
new_df = add_column(df_train.head(522), 'address_code')
print(new_df.head(10))